In [1]:
import os
import sys

import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# define data paths
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data/HIFLD')
FDIC_path = os.path.join(HIFLD_path, 'FDIC_Insured_Banks')
Prison_path = os.path.join(HIFLD_path, 'Prison_Boundaries')
UrgentCare_path = os.path.join(HIFLD_path, 'Urgent_Care_Facilities')

FDIC_file = os.path.join(FDIC_path, 'FDIC_Insured_Banks.shp')
Prison_file = os.path.join(Prison_path, 'Prison_Boundaries.shp')
UrgentCare_file = os.path.join(UrgentCare_path, 'UrgentCareFacs.shp')

In [17]:
FDIC_shapes = gpd.read_file(FDIC_file, rows=100)
FDIC_shapes.columns

Index(['OBJECTID', 'ADDRESBR', 'BRNUM', 'BRSERTYP', 'CBSABR', 'CBSANAMB',
       'CITYBR', 'CNTRYNAB', 'CNTYNAMB', 'DEPSUMBR', 'GEOCODE_CE', 'NAMEBR',
       'STALPBR', 'STCNTYBR', 'STNAMEBR', 'UNINUMBR', 'ZIPBR', 'CERT',
       'ADDRESS', 'ASSET', 'BKCLASS', 'CITY', 'CNTRYNA', 'DENOVO', 'DEPDOM',
       'NAMEFULL', 'NAMEHCR', 'REGAGNT', 'REPDTE', 'RSSDID', 'STALP', 'STCNTY',
       'STNAME', 'ZIP', 'BKMO', 'LOC_NAME', 'STATUS', 'SCORE', 'x', 'y',
       'GeocodeSou', 'STD_ADDR_B', 'STD_ADDR', 'ZIP4BR', 'geometry'],
      dtype='object')

In [31]:
FDIC_shapes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
Prison_shapes = gpd.read_file(Prison_file, rows=100)
Prison_shapes.columns

Index(['FID', 'FACILITYID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE',
       'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'SECURELVL', 'CAPACITY',
       'SHAPE_Leng', 'GlobalID', 'CreationDa', 'Creator', 'EditDate', 'Editor',
       'SHAPE_Le_1', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [30]:
Prison_shapes.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
UrgentCare_shapes = gpd.read_file(UrgentCare_file, rows=100)
UrgentCare_shapes.columns

Index(['OBJECTID', 'ID', 'NAME', 'TELEPHONE', 'ADDRESS', 'ADDRESS2', 'CITY',
       'STATE', 'ZIP', 'ZIPP4', 'COUNTY', 'FIPS', 'DIRECTIONS', 'EMERGTITLE',
       'EMERGTEL', 'EMERGEXT', 'CONTDATE', 'CONTHOW', 'GEODATE', 'GEOHOW',
       'HSIPTHEMES', 'NAICSCODE', 'NAICSDESCR', 'GEOLINKID', 'X', 'Y',
       'ST_VENDOR', 'ST_VERSION', 'GEOPREC', 'PHONELOC', 'QC_QA', 'UCAOA_ID',
       'geometry'],
      dtype='object')

In [29]:
UrgentCare_shapes.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Full Address String

In [21]:
def full_address(df):
    if 'STATE' in df.columns:
        df['Full_Address'] = df[['NAME', 'ADDRESS', 'CITY', 'STATE']].agg(', '.join, axis=1) + ' ' + df['ZIP']
    else:
        df['Full_Address'] = df[['STD_ADDR_B', 'CITY', 'STNAME']].agg(', '.join, axis=1) + ' ' + df['ZIP']
        
    return df

In [7]:
filenames = ['FDIC', 'Prison', 'UrgentCare']
dfs = [FDIC_shapes, Prison_shapes, UrgentCare_shapes]

new_df = dict()
for df, fname in zip(dfs, filenames):
    new_df[fname] = full_address(df)

# Calculate Lat/Lon

In [11]:
# convert to CRS 'EPSG:4326'

def convert_EPSG4326(dict):
    """
    dict from full_address
    """
    for fname in dict:
        dict[fname] = dict[fname].to_crs("EPSG:4326")

    return dict

In [12]:
new_df = convert_EPSG4326(new_df)

In [26]:
def get_centroid(dict):
    """
    Returns Point object or a coordinate tuple (x, y)
    """
    for fname in dict:
        if 'x' in dict[fname].columns:
            dict[fname] = dict[fname].rename(columns={'x': 'Lon', 'y': 'Lat'})
        else:
            dict[fname]['centroid'] = dict[fname]['geometry'].centroid
            dict[fname]['Lon'] = dict[fname]['geometry'].centroid.x
            dict[fname]['Lat'] = dict[fname]['geometry'].centroid.y
        
    return dict

In [27]:
new_df = get_centroid(new_df)

C:\Users\bchan\AppData\Local\Temp\ipykernel_10108\1505800128.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['centroid'] = dict[fname]['geometry'].centroid
C:\Users\bchan\AppData\Local\Temp\ipykernel_10108\1505800128.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['Lon'] = dict[fname]['geometry'].centroid.x
C:\Users\bchan\AppData\Local\Temp\ipykernel_10108\1505800128.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['Lat'] = dict[fname]['geometry'].centroid.y
C:\Users\bchan\AppData\Local\Temp\ipykernel_10108\1505800128.py:1

In [28]:
# check centroids
new_df['Prison']

,FID,FACILITYID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,...,Creator,EditDate,Editor,SHAPE_Le_1,SHAPE_Area,geometry,Full_Address,centroid,Lon,Lat
0,1,10002798,MIDLAND COUNTY CENTRAL DETENTION CENTER,400 S MAIN ST,MIDLAND,TX,79701,-999,(432) 688-4745,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,863.871076,2.579058e-06,"POLYGON ((-102.07687 31.99381, -102.07442 31.9...","MIDLAND COUNTY CENTRAL DETENTION CENTER, 400 S...",POINT (-102.07542 31.99396),-102.075419,31.993959
1,2,10002796,BARBARA CULVER JUVENILE DETENTION CENTER,3800 N LAMESA RD,MIDLAND,TX,79701,-999,(432) 688-4541,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,571.238626,9.897645e-07,"POLYGON ((-102.07655 32.03859, -102.07622 32.0...","BARBARA CULVER JUVENILE DETENTION CENTER, 3800...",POINT (-102.07635 32.03787),-102.076346,32.037874
2,3,10002836,SAN JACINTO COUNTY JAIL,75 W CEDAR AVE,COLDSPRING,TX,77331,-999,(936) 653-4367,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,285.360222,3.075715e-07,"POLYGON ((-95.12992 30.59128, -95.12955 30.591...","SAN JACINTO COUNTY JAIL, 75 W CEDAR AVE, COLDS...",POINT (-95.12961 30.59095),-95.129614,30.590948
3,4,10003033,YELLOW MEDICINE COUNTY JAIL,415 9TH AVENUE,GRANITE FALLS,MN,56241,-999,(320) 313-3049,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,304.118316,1.900107e-07,"POLYGON ((-95.54458 44.80976, -95.54434 44.809...","YELLOW MEDICINE COUNTY JAIL, 415 9TH AVENUE, G...",POINT (-95.54461 44.80948),-95.544607,44.809476
4,5,10006288,NMJC THIEF RIVER FALLS SATELLITE HOME,118 N. SPRUCE AVENUE,THIEF RIVER FALLS,MN,56701,-999,(218) 681-3484,LOCAL,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,85.538141,2.457488e-08,"POLYGON ((-96.16240 48.11734, -96.16221 48.117...","NMJC THIEF RIVER FALLS SATELLITE HOME, 118 N. ...",POINT (-96.16230 48.11727),-96.162300,48.117274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,10003361,WOOD COUNTY JUSTICE CENTER,1960 E GYPSY LANE RD,BOWLING GREEN,OH,43402,-999,(419) 354-7744,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,1021.188460,1.328222e-06,"POLYGON ((-83.61322 41.35528, -83.61296 41.355...","WOOD COUNTY JUSTICE CENTER, 1960 E GYPSY LANE ...",POINT (-83.61261 41.35462),-83.612607,41.354619
96,97,10002617,CENTRAL UTAH CORRECTIONAL FACILITY,255 E 300 N ST,GUNNISON,UT,84634,-999,(435) 528-6000,STATE,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,3190.701838,2.768990e-05,"POLYGON ((-111.81211 39.16849, -111.81068 39.1...","CENTRAL UTAH CORRECTIONAL FACILITY, 255 E 300 ...",POINT (-111.81150 39.16624),-111.811498,39.166243
97,98,10003453,MILL CREEK YOUTH CENTER,790 W 12TH ST,OGDEN,UT,84404,-999,(801) 334-0210,STATE,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,1021.255219,2.657356e-06,"POLYGON ((-111.99853 41.24914, -111.99755 41.2...","MILL CREEK YOUTH CENTER, 790 W 12TH ST, OGDEN,...",POINT (-111.99821 41.24826),-111.998206,41.248260
98,99,10000985,ATHENS COUNTY JAIL,13 W WASHINGTON ST,ATHENS,OH,45701,-999,(740) 593-6633,COUNTY,...,HostedByHIFLD,2022-01-07,HostedByHIFLD,138.035169,6.310239e-08,"POLYGON ((-82.10191 39.32912, -82.10172 39.329...","ATHENS COUNTY JAIL, 13 W WASHINGTON ST, ATHENS...",POINT (-82.10183 39.32895),-82.101833,39.328949
